In [1]:
from pathlib import Path
print((Path("/kaggle/input/celeba-dataset")/"img_align_celeba").exists())
print((Path("/kaggle/input/celeba-dataset")/"list_attr_celeba.csv").exists())
print((Path("/kaggle/input/celeba-dataset")/"list_eval_partition.csv").exists())
print((Path("/kaggle/input/hat-or-no-hat-that-is-the-question-fall-25/test_set/test_set")).exists())


True
True
True
True


In [2]:
# ===== 0) Setup =====
from pathlib import Path
import numpy as np, pandas as pd, tensorflow as tf

tf.keras.mixed_precision.set_global_policy("mixed_float16")

COMP_ROOT = Path("/kaggle/input/hat-or-no-hat-that-is-the-question-fall-25")
TEST_DIR  = COMP_ROOT/"test_set"/"test_set"          # nested folder

CELEBA_ROOT = Path("/kaggle/input/celeba-dataset")
ATTR   = CELEBA_ROOT/"list_attr_celeba.csv"
SPLIT  = CELEBA_ROOT/"list_eval_partition.csv"
IMG_DIR= CELEBA_ROOT/"img_align_celeba"/"img_align_celeba"

WORK_DIR = Path("/kaggle/working"); WORK_DIR.mkdir(exist_ok=True)
IMG_SIZE = (224,224); BATCH = 64; SEED = 42
tf.keras.utils.set_random_seed(SEED)


2025-10-27 01:54:21.427077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761530061.885123      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761530062.009357      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# ===== 1) Labels, paths, 5k subset =====
attrs = pd.read_csv(ATTR)
if "image_id" not in attrs.columns:
    attrs.rename(columns={attrs.columns[0]:"image_id"}, inplace=True)
parts = pd.read_csv(SPLIT); parts.columns = ["image_id","split"]

df = attrs.merge(parts, on="image_id")
df["label"] = (df["Wearing_Hat"]==1).astype("int8")          # 1=Hat, 0=No Hat
df["filepath"] = df["image_id"].apply(lambda x: str((IMG_DIR/x).as_posix()))

train_full = df[df["split"]==0].reset_index(drop=True)

TARGET = 10000
pos, neg = train_full[train_full.label==1], train_full[train_full.label==0]
n_pos = min(len(pos), TARGET//2); n_neg = TARGET - n_pos
sub5k = pd.concat([
    pos.sample(n=n_pos, random_state=SEED),
    neg.sample(n=n_neg, random_state=SEED)
]).sample(frac=1.0, random_state=SEED).reset_index(drop=True)

# keep a manifest
(sub5k_path := WORK_DIR/"celeba_manifest_5k.csv")
sub5k.to_csv(sub5k_path, index=False)


In [4]:
# ===== 2) Split, class weights, datasets =====
from sklearn.model_selection import train_test_split

train_5k, val_5k = train_test_split(
    sub5k, test_size=0.1, stratify=sub5k["label"], random_state=SEED
)

# optional class weights (only if imbalanced)
pos_ct = int((train_5k.label == 1).sum())
neg_ct = int((train_5k.label == 0).sum())
total  = pos_ct + neg_ct
imbalance_ratio = min(pos_ct, neg_ct) / max(pos_ct, neg_ct)
class_weight = {
    0: total/(2.0*neg_ct),
    1: total/(2.0*pos_ct),
} if imbalance_ratio < 0.7 else None

def make_ds(frame, training):
    paths = frame["filepath"].values
    labels = frame["label"].values.astype("float32")
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    def _load(p, y):
        x = tf.io.read_file(p); x = tf.image.decode_jpeg(x, 3)
        x = tf.image.resize(x, IMG_SIZE)
        if training:
            x = tf.image.random_flip_left_right(x)
            x = tf.image.random_brightness(x, 0.15)
            x = tf.image.random_contrast(x, 0.8, 1.2)
            x = tf.image.random_saturation(x, 0.8, 1.2)
            crop = tf.image.random_crop(
                tf.image.resize_with_pad(x, IMG_SIZE[0]+16, IMG_SIZE[1]+16),
                size=(IMG_SIZE[0], IMG_SIZE[1], 3)
            )
            x = tf.where(tf.random.uniform([])<0.5, crop, x)
        return tf.clip_by_value(x/255.0, 0., 1.), y
    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE)
    if training: ds = ds.shuffle(4096, seed=SEED)
    return ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)

train_ds = make_ds(train_5k, True)
val_ds   = make_ds(val_5k,   False)


I0000 00:00:1761530083.973319      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1761530083.973998      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [5]:
# ===== 3) Model and training =====
from tensorflow import keras as K

base = K.applications.EfficientNetB0(include_top=False, weights=None,
                                     input_shape=(*IMG_SIZE,3))
inp = K.Input((*IMG_SIZE,3))
x = base(inp, training=True)
x = K.layers.GlobalAveragePooling2D()(x)
x = K.layers.Dropout(0.3)(x)
out = K.layers.Dense(1, activation="sigmoid", dtype="float32")(x)   # FP32 head
model = K.Model(inp, out)

model.compile(optimizer=K.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy", K.metrics.AUC(name="auc")])

ckpt = K.callbacks.ModelCheckpoint(str(WORK_DIR/"best.h5"),
                                   save_best_only=True, monitor="val_loss", mode="min")
es   = K.callbacks.EarlyStopping(patience=6, restore_best_weights=True,
                                 monitor="val_loss", mode="min")
rlrp = K.callbacks.ReduceLROnPlateau(patience=2, factor=0.2)

model.fit(train_ds, validation_data=val_ds, epochs=20,
          callbacks=[ckpt, es, rlrp], class_weight=class_weight, verbose=1)


Epoch 1/20


I0000 00:00:1761530189.223778      61 service.cc:148] XLA service 0x7d5434002930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761530189.225384      61 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1761530189.225407      61 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1761530195.498524      61 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1761530247.332733      61 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


141/141 ━━━━━━━━━━━━━━━━━━━━ 295s 963ms/step - accuracy: 0.6945 - auc: 0.7519 - loss: 0.6254 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 1.5555 - learning_rate: 0.0010
Epoch 2/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 26s 137ms/step - accuracy: 0.9114 - auc: 0.9665 - loss: 0.2321 - val_accuracy: 0.5000 - val_auc: 0.5010 - val_loss: 2.3898 - learning_rate: 0.0010
Epoch 3/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 27s 140ms/step - accuracy: 0.9409 - auc: 0.9840 - loss: 0.1580 - val_accuracy: 0.5000 - val_auc: 0.8460 - val_loss: 1.3885 - learning_rate: 0.0010
Epoch 4/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 26s 137ms/step - accuracy: 0.9558 - auc: 0.9880 - loss: 0.1302 - val_accuracy: 0.6840 - val_auc: 0.9435 - val_loss: 0.8386 - learning_rate: 0.0010
Epoch 5/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 27s 140ms/step - accuracy: 0.9697 - auc: 0.9934 - loss: 0.0912 - val_accuracy: 0.9130 - val_auc: 0.9771 - val_loss: 0.2404 - learning_rate: 0.0010
Epoch 6/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 25s 134ms/step - accuracy: 0.9749 -

In [6]:
# ===== 4) Threshold on validation =====
from sklearn.metrics import roc_curve
val_probs = model.predict(val_ds, verbose=0).ravel()
val_true  = val_5k["label"].to_numpy().astype(int)
fpr, tpr, thr = roc_curve(val_true, val_probs)
tau = float(thr[(tpr - fpr).argmax()])


In [7]:
# ===== 5) Test inference (robust) + submission =====
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

test_paths = sorted(Path(TEST_DIR).rglob("*.jpg"))

# split into TF-decodable vs fallback
good, bad = [], []
for p in test_paths:
    try:
        _ = tf.io.decode_image(tf.io.read_file(str(p)), channels=3, expand_animations=False)
        good.append(p)
    except Exception:
        bad.append(p)

def load_for_infer(p):
    b = tf.io.read_file(p)
    x = tf.io.decode_image(b, channels=3, expand_animations=False)
    x.set_shape([None, None, 3])
    x = tf.image.resize(x, IMG_SIZE)
    return x/255.0

infer_ds = (tf.data.Dataset.from_tensor_slices([str(p) for p in good])
            .map(load_for_infer, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(64).prefetch(tf.data.AUTOTUNE))
probs_good = model.predict(infer_ds, verbose=0).ravel()

arr_bad = []
for p in bad:
    im = Image.open(p).convert("RGB").resize(IMG_SIZE, Image.BILINEAR)
    arr_bad.append(np.asarray(im, dtype=np.float32)/255.0)
probs_bad = model.predict(np.stack(arr_bad), batch_size=8, verbose=0).ravel() if bad else np.array([])

proba_map = {p: pr for p, pr in zip(good, probs_good)}
proba_map.update({p: pr for p, pr in zip(bad, probs_bad)})
probs = np.array([proba_map[p] for p in test_paths], dtype=np.float32)

labels = np.where(probs >= tau, "Hat", "No Hat")
ids = [p.stem for p in test_paths]

pd.DataFrame({"id": ids, "class": labels}).to_csv(WORK_DIR/"submission.csv", index=False)


In [8]:
!ls -lh /kaggle/working/

total 51M
-rw-r--r-- 1 root root  47M Oct 27 02:05 best.h5
-rw-r--r-- 1 root root 2.0M Oct 27 01:54 celeba_manifest_5k.csv
---------- 1 root root 1.4M Oct 27 02:09 __notebook__.ipynb
-rw-r--r-- 1 root root  56K Oct 27 02:09 submission.csv
